In [2]:
import pandas as pd
import numpy  as np

In [3]:
pd.options.display.max_columns = None

# 00 - ZIP Codes

In [4]:
df_zipcodes_raw = pd.read_excel('../data/raw/PLZ_Verzeichnis-20221005.xls')

df_zipcodes_raw.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,PLZ,Ort,Bundesland,gültig ab,gültig bis,NamePLZTyp,intern_extern,adressierbar,Postfach
0,1000,Wien,W,01.08.2009,NaN,PLZ-Postfach,extern,Nein,Ja
1,1004,Wien,W,01.01.1966,NaN,InteressentenPLZ,extern,Nein,Ja
2,1006,Wien,W,01.02.2015,NaN,PLZ-Postfach,extern,Nein,Ja
3,1010,Wien,W,01.01.1966,NaN,PLZ-Adressierung,extern,Ja,Ja
4,1011,Wien Postfach,W,01.01.1966,NaN,PLZ-Postfach,extern,Nein,Ja


In [5]:
df_zipcodes_raw = df_zipcodes_raw.rename(columns=lambda x:x.strip())

In [6]:
df_zipcodes = df_zipcodes_raw[['PLZ', 'Bundesland']].copy()
df_zipcodes.tail()

,PLZ,Bundesland
2516,9974,T
2517,9981,T
2518,9990,T
2519,9991,T
2520,9992,T


In [7]:
df_zipcodes.Bundesland.unique()

array(['W', 'N', 'B', 'O', 'Sa', 'T', 'V', 'St', 'K'], dtype=object)

In [8]:
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('W'), 'Bundesland']  = 'Wien'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('N'), 'Bundesland']  = 'Niederoesterreich'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('B'), 'Bundesland']  = 'Burgenland'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('O'), 'Bundesland']  = 'Oberoesterreich'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('Sa'),'Bundesland']  = 'Salzburg'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('T'), 'Bundesland']  = 'Tirol'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('V'), 'Bundesland']  = 'Vorarlberg'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('St'),'Bundesland']  = 'Steiermark'
df_zipcodes.loc[df_zipcodes.Bundesland.str.startswith('K'), 'Bundesland']  = 'Kärnten'

In [9]:
df_zipcodes.Bundesland.unique()

array(['Wien', 'Niederoesterreich', 'Burgenland', 'Oberoesterreich',
       'Salzburg', 'Tirol', 'Vorarlberg', 'Steiermark', 'Kärnten'],
      dtype=object)

In [10]:
df_zipcodes.isna().any()

PLZ           False
Bundesland    False
dtype: bool

# 01 - Cleaning the private hospital dataset

In [11]:
df_private_hospitals_raw = pd.read_excel('../data/raw/liste_der_selbststaendigen_ambulatorien_in_oesterreich_(datenbankabruf_03.10.2022).xlsx',
                                         skiprows=3)

This dataset contains two parts 
- general information (city, address...) 
- True/False informations about the type of private hospital (HNO, Dialyse)

We will split data frames in these two parts and apply an individual preprocessing approach 

### First part (Hospital Informations)

In [12]:
df_private_hospitals_info = df_private_hospitals_raw.loc[:, : 'Ärztliche Leitung'].copy()

df_private_hospitals_info.head()

,Bundesland,A-Nr.,Bezeichnung,Post-\nleitzahl,Ortsbe-\nzeichnung,Strasse,Telefon-Nr.,Fax-Nr.,Homepage,Ärztliche Leitung
0,Burgenland,A10101,Ambulatorium für medizinische und chemische La...,7000,Eisenstadt,Neusiedlerstraße 35-37,02682 66002,02682 75879,http://daslabor-eisenstadt.at,Dr. Peter Ozabal
1,Burgenland,A10102,Österreichische Gesundheitskasse - Mein Gesund...,7000,Eisenstadt,Siegfried Marcus-Straße 5,05 0766 131960,02682 608 1963,http://www.gesundheitskasse.at,Dr. Lukas Gallei
2,Burgenland,A10103,Röntgenambulatorium Eisenstadt Gesellschaft mbH,7000,Eisenstadt,Betriebsstraße 8,02682 63936,NaN,http://www.roentgen-eisenstadt.at,Dr. Wolfram Kluger
3,Burgenland,A10104,Österreichische Gesundheitskasse - Mein Zahnge...,7000,Eisenstadt,Siegfried Marcus-Straße 5,05 0766 131980,02682 608 1963,www.gesundheitskasse.at,Dr.in Renate Schaudy
4,Burgenland,A10105,Ambulatroium für Kinder- und Jugendpsychiatrie,7000,Eisenstadt,Franz Liszt Straße 1,057979 21 100,057979 21 110,http://www.psychosozialerdienst.at,Dr. Roland Grassl


In [13]:
df_private_hospitals_info.columns

Index(['Bundesland', 'A-Nr.', 'Bezeichnung', 'Post-\nleitzahl',
       'Ortsbe-\nzeichnung', 'Strasse', 'Telefon-Nr.', 'Fax-Nr.', 'Homepage',
       'Ärztliche Leitung'],
      dtype='object')

In [14]:
df_private_hospitals_info = df_private_hospitals_info.drop(columns=['Bezeichnung', 'Fax-Nr.', 'A-Nr.', 'Telefon-Nr.', 'Homepage', 'Ortsbe-\nzeichnung', 'Strasse', 'Ärztliche Leitung'])

df_private_hospitals_info.head()

,Bundesland,Post-\nleitzahl
0,Burgenland,7000
1,Burgenland,7000
2,Burgenland,7000
3,Burgenland,7000
4,Burgenland,7000


In [15]:
df_private_hospitals_info = df_private_hospitals_info.rename(columns={'Post-\nleitzahl': 'PLZ'})

df_private_hospitals_info.head()

,Bundesland,PLZ
0,Burgenland,7000
1,Burgenland,7000
2,Burgenland,7000
3,Burgenland,7000
4,Burgenland,7000


In [16]:
# Strip every column of whitespace
df_private_hospitals_info = df_private_hospitals_info.rename(columns=lambda x: x.strip())

df_private_hospitals_info.head()

,Bundesland,PLZ
0,Burgenland,7000
1,Burgenland,7000
2,Burgenland,7000
3,Burgenland,7000
4,Burgenland,7000


In [17]:
df_private_hospitals_info.isnull().any()

Bundesland    False
PLZ           False
dtype: bool

In [18]:
df_private_hospitals_info.isnull().sum(axis=0)

Bundesland    0
PLZ           0
dtype: int64

### Checking invalid zip codes

In [19]:
~df_private_hospitals_info.PLZ.isin(df_zipcodes.PLZ).any()

False

### Second part (Hospital Task Area)

In [20]:
df_private_hospitals_task_area = df_private_hospitals_raw.loc[:, 'Allergien (Immunologie, Haut, Lunge, ...)':].copy()

df_private_hospitals_task_area.head()

,"Allergien (Immunologie, Haut, Lunge, ...)","ambulante Rehabilitation PSY, NEURO und ONKO",Arbeitsmedizin / e-Impfpass,"Augen, Sehschule",Bildgebende Verfahren (inkl. Nuklearmedizin),"Blutspende, Blutbanken",Chirurgie,Dialyse/Nieren,"Fertilitätsstörung, Schwangerenbetreuung, InVitro Fertilisation","Frauenmedizin, Hormondiagnostik, klimakterische Beschwerden","Genetik, Andrologie","Hals-, Nasen- und Ohrenkrankheiten",Haut- und Geschlechtskrankheiten,"Intern, Herz/Kreislauf, Lunge","Komplimentärmedizin, Naturheilverfahren",Labordiagnostik,Mehrzweckambulatorium,"Neurologie, Neurophysiologie, Psychiatrie","Orthopädie, Sportmedizin",Physikalische Medizin (inkl. Balneotherapie),"Prävention, Vorsorge",Primärversorgungseinheit,"Psychiatrie, Psychotherapie, psychosoziale Einrichtungen, Sucht",Schmerzbehandlung,Sonstiges,"Stoffwechselstörungen, Diät, Diabetes","Verhaltenstörung, Behinderung, Hörschädigung",Zahn/Kiefer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ü,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ü,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,ü,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ü
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ü,NaN,NaN,NaN,NaN,NaN


In [21]:
df_private_hospitals_task_area = df_private_hospitals_task_area.replace({np.nan: 0, 'ü': 1})

df_private_hospitals_task_area

,"Allergien (Immunologie, Haut, Lunge, ...)","ambulante Rehabilitation PSY, NEURO und ONKO",Arbeitsmedizin / e-Impfpass,"Augen, Sehschule",Bildgebende Verfahren (inkl. Nuklearmedizin),"Blutspende, Blutbanken",Chirurgie,Dialyse/Nieren,"Fertilitätsstörung, Schwangerenbetreuung, InVitro Fertilisation","Frauenmedizin, Hormondiagnostik, klimakterische Beschwerden","Genetik, Andrologie","Hals-, Nasen- und Ohrenkrankheiten",Haut- und Geschlechtskrankheiten,"Intern, Herz/Kreislauf, Lunge","Komplimentärmedizin, Naturheilverfahren",Labordiagnostik,Mehrzweckambulatorium,"Neurologie, Neurophysiologie, Psychiatrie","Orthopädie, Sportmedizin",Physikalische Medizin (inkl. Balneotherapie),"Prävention, Vorsorge",Primärversorgungseinheit,"Psychiatrie, Psychotherapie, psychosoziale Einrichtungen, Sucht",Schmerzbehandlung,Sonstiges,"Stoffwechselstörungen, Diät, Diabetes","Verhaltenstörung, Behinderung, Hörschädigung",Zahn/Kiefer
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
935,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
936,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [22]:
df_private_hospitals_task_area = df_private_hospitals_task_area.rename(columns=lambda x: x.strip())

In [23]:
for column in df_private_hospitals_task_area.columns:
    print(column)

Allergien (Immunologie, Haut, Lunge, ...)
ambulante Rehabilitation PSY, NEURO und ONKO
Arbeitsmedizin / e-Impfpass
Augen, Sehschule
Bildgebende Verfahren (inkl. Nuklearmedizin)
Blutspende, Blutbanken
Chirurgie
Dialyse/Nieren
Fertilitätsstörung, Schwangerenbetreuung, InVitro Fertilisation
Frauenmedizin, Hormondiagnostik, klimakterische Beschwerden
Genetik, Andrologie
Hals-, Nasen- und Ohrenkrankheiten
Haut- und Geschlechtskrankheiten
Intern, Herz/Kreislauf, Lunge
Komplimentärmedizin, Naturheilverfahren
Labordiagnostik
Mehrzweckambulatorium
Neurologie, Neurophysiologie, Psychiatrie
Orthopädie, Sportmedizin
Physikalische Medizin (inkl. Balneotherapie)
Prävention, Vorsorge
Primärversorgungseinheit
Psychiatrie, Psychotherapie, psychosoziale Einrichtungen, Sucht
Schmerzbehandlung
Sonstiges
Stoffwechselstörungen, Diät, Diabetes
Verhaltenstörung, Behinderung, Hörschädigung
Zahn/Kiefer


In [24]:
df_private_hospitals_task_area = df_private_hospitals_task_area.rename(columns={'Allergien (Immunologie, Haut, Lunge, ...)': 'Allergien',
                                                                                'ambulante Rehabilitation PSY, NEURO und ONKO': 'Ambulante_Rehabilitation',
                                                                                'Arbeitsmedizin / e-Impfpass': 'Arbeitsmedizin',
                                                                                'Augen, Sehschule': 'Augenarzt_Sehschule',
                                                                                'Bildgebende Verfahren (inkl. Nuklearmedizin)': 'Bildgebende_Verfahren',
                                                                                'Blutspende, Blutbanken': 'Blutspende_Blutbanken',
                                                                                'Dialyse/Nieren': 'Dialyse',
                                                                                'Fertilitätsstörung, Schwangerenbetreuung, InVitro Fertilisation': 'Schwangerenbetreuung_Fertilitätsbehandlungen',
                                                                                'Frauenmedizin, Hormondiagnostik, klimakterische Beschwerden': 'Frauenmedizn',
                                                                                'Genetik, Andrologie': 'Andrologie',
                                                                                'Hals-, Nasen- und Ohrenkrankheiten': 'HNO',
                                                                                'Haut- und Geschlechtskrankheiten': 'Dermatologie',
                                                                                'Intern, Herz/Kreislauf, Lunge': 'Herz_Lunge',
                                                                                'Komplimentärmedizin, Naturheilverfahren': 'Komplimentärmedizin',
                                                                                'Neurologie, Neurophysiologie, Psychiatrie': 'Neurologie_Neurophysiologie_Psychatrie',
                                                                                'Orthopädie, Sportmedizin': 'Orthopädie_Sportmedizin',
                                                                                'Physikalische Medizin (inkl. Balneotherapie)': 'Physikalische_Medizin',
                                                                                'Prävention, Vorsorge': 'Praevention_Vorsorge',
                                                                                'Psychiatrie, Psychotherapie, psychosoziale Einrichtungen, Sucht': 'Psychosoziale_Einrichtung',
                                                                                'Stoffwechselstörungen, Diät, Diabetes': 'Stoffwechselstörungen',
                                                                                'Verhaltenstörung, Behinderung, Hörschädigung': 'Verhaltensstoerung_Behinderung',
                                                                                'Zahn/Kiefer': 'Zahn_Kiefer'})

df_private_hospitals_task_area.head()

,Allergien,Ambulante_Rehabilitation,Arbeitsmedizin,Augenarzt_Sehschule,Bildgebende_Verfahren,Blutspende_Blutbanken,Chirurgie,Dialyse,Schwangerenbetreuung_Fertilitätsbehandlungen,Frauenmedizn,Andrologie,HNO,Dermatologie,Herz_Lunge,Komplimentärmedizin,Labordiagnostik,Mehrzweckambulatorium,Neurologie_Neurophysiologie_Psychatrie,Orthopädie_Sportmedizin,Physikalische_Medizin,Praevention_Vorsorge,Primärversorgungseinheit,Psychosoziale_Einrichtung,Schmerzbehandlung,Sonstiges,Stoffwechselstörungen,Verhaltensstoerung_Behinderung,Zahn_Kiefer
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


## Final join

In [25]:
df_private_hospitals = df_private_hospitals_info.join(df_private_hospitals_task_area)

df_private_hospitals.head()

,Bundesland,PLZ,Allergien,Ambulante_Rehabilitation,Arbeitsmedizin,Augenarzt_Sehschule,Bildgebende_Verfahren,Blutspende_Blutbanken,Chirurgie,Dialyse,Schwangerenbetreuung_Fertilitätsbehandlungen,Frauenmedizn,Andrologie,HNO,Dermatologie,Herz_Lunge,Komplimentärmedizin,Labordiagnostik,Mehrzweckambulatorium,Neurologie_Neurophysiologie_Psychatrie,Orthopädie_Sportmedizin,Physikalische_Medizin,Praevention_Vorsorge,Primärversorgungseinheit,Psychosoziale_Einrichtung,Schmerzbehandlung,Sonstiges,Stoffwechselstörungen,Verhaltensstoerung_Behinderung,Zahn_Kiefer
0,Burgenland,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Burgenland,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Burgenland,7000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Burgenland,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Burgenland,7000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
df_private_hospitals.to_csv('../data/cleaned/priave_ambulatorien.csv', index=False)

# 02 - Cleaning the KA-Verzeichnis dataset

In [27]:
df_hospitals_raw = pd.read_excel('../data/raw/KA-Verzeichnis 2022-10-07.xlsx', sheet_name='Standorte Akutversorgung', skiprows=2)

df_hospitals_raw.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,01 Anästhesiologie und Intensivmedizin/Intensivversorgung,02/1 Kinder- und Jugendheilkunde,02/2 Kinder- und Jugendchirurgie,02/3 Kinder- und Jugendpsychiatrie,03 Chirurgie,04 Neurochirurgie,05 Innere Medizin,06 Frauenheilkunde und Geburtshilfe,07 Neurologie,08 Psychiatrie,09 Haut- und Geschlechtskrankheiten,10 Augenheilkunde,"11 Hals-, Nasen- und Ohrenheilkunde",12 Urologie,13 Plastische Chirurgie,15 Orthopädie und Traumatologie,"16 Mund-, Kiefer- und Gesichtschirurgie","17 Zahn-, Mund- und Kieferheilkunde",18 Strahlentherapie-Radioonkologie,19 Nuklearmedizinische Therapie,50 Gemischter Belag,61/1 Psychosomatik/Erwachsene,61/2 Psychosomatik/KJ,62 Akutgeriatrie/Remobilisation,63 Remobilisation/Nachsorge,64/1 Palliativmedizin/Erwachsene,64/2 Palliativmedizin/KJ
0,K102,Eisenstadt BBR KH,geöffnet,10101 Eisenstadt,7000 Eisenstadt,Johannes von Gott-Platz 1,30.0,16.0,NaN,NaN,36.0,NaN,90.0,25.0,34.0,41.0,NaN,NaN,13.0,NaN,NaN,48.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,4.0,NaN
1,K104,Güssing LKH,geöffnet,10405 Güssing,7540 Güssing,Grazer Straße 13,10.0,NaN,NaN,NaN,39.0,NaN,37.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,K105,Kittsee LKH,geöffnet,10711 Kittsee,2421 Kittsee,Hauptplatz 3,10.0,NaN,NaN,NaN,30.0,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K106,Oberpullendorf LKH,geöffnet,10816 Oberpullendorf,7350 Oberpullendorf,Spitalstraße 32,12.0,NaN,NaN,NaN,29.0,NaN,49.0,23.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,K107,Oberwart LKH,geöffnet,10917 Oberwart,7400 Oberwart,Dornburggasse 80,22.0,21.0,NaN,NaN,32.0,NaN,61.0,25.0,38.0,NaN,NaN,NaN,4.0,31.0,NaN,47.0,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,8.0,NaN


Again, we'll approach this with a two step data cleaning method <br>
- Hospital info
- How many beds?

In [28]:
df_hospitals_info = df_hospitals_raw.iloc[:, :5].copy()

df_hospitals_info.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,K102,Eisenstadt BBR KH,geöffnet,10101 Eisenstadt,7000 Eisenstadt
1,K104,Güssing LKH,geöffnet,10405 Güssing,7540 Güssing
2,K105,Kittsee LKH,geöffnet,10711 Kittsee,2421 Kittsee
3,K106,Oberpullendorf LKH,geöffnet,10816 Oberpullendorf,7350 Oberpullendorf
4,K107,Oberwart LKH,geöffnet,10917 Oberwart,7400 Oberwart


In [29]:
df_hospitals_info = df_hospitals_info.rename(columns={'Unnamed: 1': 'Name', 'Unnamed: 4': 'PLZ_Region'})

df_hospitals_info.head()

,Unnamed: 0,Name,Unnamed: 2,Unnamed: 3,PLZ_Region
0,K102,Eisenstadt BBR KH,geöffnet,10101 Eisenstadt,7000 Eisenstadt
1,K104,Güssing LKH,geöffnet,10405 Güssing,7540 Güssing
2,K105,Kittsee LKH,geöffnet,10711 Kittsee,2421 Kittsee
3,K106,Oberpullendorf LKH,geöffnet,10816 Oberpullendorf,7350 Oberpullendorf
4,K107,Oberwart LKH,geöffnet,10917 Oberwart,7400 Oberwart


In [30]:
df_hospitals_info = df_hospitals_info.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3'])

df_hospitals_info.head()

,Name,PLZ_Region
0,Eisenstadt BBR KH,7000 Eisenstadt
1,Güssing LKH,7540 Güssing
2,Kittsee LKH,2421 Kittsee
3,Oberpullendorf LKH,7350 Oberpullendorf
4,Oberwart LKH,7400 Oberwart


In [31]:
df_hospitals_info['PLZ'] = df_hospitals_info.PLZ_Region.str[:4]

df_hospitals_info.head()

,Name,PLZ_Region,PLZ
0,Eisenstadt BBR KH,7000 Eisenstadt,7000
1,Güssing LKH,7540 Güssing,7540
2,Kittsee LKH,2421 Kittsee,2421
3,Oberpullendorf LKH,7350 Oberpullendorf,7350
4,Oberwart LKH,7400 Oberwart,7400


In [32]:
df_hospitals_info.query('PLZ_Region.str.endswith("Salzburg")')

,Name,PLZ_Region,PLZ
70,Oberndorf KH,5110 Oberndorf bei Salzburg,5110
74,Salzburg LKA,5020 Salzburg,5020
75,Salzburg UKH,5020 Salzburg,5020
76,Salzburg Wehrle-DIA PKL,5020 Salzburg,5020
77,Salzburg BBR KH,5010 Salzburg,5010
78,Salzburg LNKL / Salzburg (K528),5020 Salzburg,5020
88,Salzburg PKL,5020 Salzburg,5020


Now we'll create the Bundesland column based on the PLZ, working with the zipcodes data frame we used above

The goal: Create a dictionary with 'PLZ':'Bundesland' to map every zipcode in the hospital data frame to their Bundesland <br>
First, we need a True/False Series Mask only containig every PLZ mentioned in the hospital df

In [33]:
mask = df_zipcodes.PLZ.astype(str).isin(df_hospitals_info.PLZ)

mask

0       False
1       False
2       False
3        True
4       False
        ...  
2516    False
2517    False
2518    False
2519    False
2520    False
Name: PLZ, Length: 2521, dtype: bool

Now we masked the zipcodes df exclusively with zipcodes that appear in our hospital set

Now we convert everything to a pandas Series which will be converted into a dictionary 

In [34]:
df_zipcodes[mask].set_index('PLZ').Bundesland

PLZ
1010       Wien
1020       Wien
1030       Wien
1050       Wien
1060       Wien
         ...   
9560    Kärnten
9620    Kärnten
9640    Kärnten
9800    Kärnten
9900      Tirol
Name: Bundesland, Length: 143, dtype: object

In [35]:
mentioned_zipcodes_in_df_hospitals = df_zipcodes[mask].set_index('PLZ').Bundesland.to_dict()

Finally, we can map the values and create a new column in our dataset containing the city

In [36]:
df_hospitals_info.PLZ.astype(int).map(mentioned_zipcodes_in_df_hospitals)

0      Burgenland
1      Burgenland
2      Burgenland
3      Burgenland
4      Burgenland
          ...    
180          Wien
181          Wien
182          Wien
183          Wien
184          Wien
Name: PLZ, Length: 185, dtype: object

In [37]:
df_hospitals_info['Bundesland'] = df_hospitals_info.PLZ.astype(int).map(mentioned_zipcodes_in_df_hospitals)

df_hospitals_info.head()

,Name,PLZ_Region,PLZ,Bundesland
0,Eisenstadt BBR KH,7000 Eisenstadt,7000,Burgenland
1,Güssing LKH,7540 Güssing,7540,Burgenland
2,Kittsee LKH,2421 Kittsee,2421,Burgenland
3,Oberpullendorf LKH,7350 Oberpullendorf,7350,Burgenland
4,Oberwart LKH,7400 Oberwart,7400,Burgenland


In [38]:
df_hospitals_info = df_hospitals_info.drop(columns=['PLZ_Region'])

df_hospitals_info.head()

,Name,PLZ,Bundesland
0,Eisenstadt BBR KH,7000,Burgenland
1,Güssing LKH,7540,Burgenland
2,Kittsee LKH,2421,Burgenland
3,Oberpullendorf LKH,7350,Burgenland
4,Oberwart LKH,7400,Burgenland


Lets focus on the amount of beds in every hospitals

In [39]:
df_hospitals_beds = df_hospitals_raw.iloc[:, 6:].copy()

df_hospitals_beds.head()

,01 Anästhesiologie und Intensivmedizin/Intensivversorgung,02/1 Kinder- und Jugendheilkunde,02/2 Kinder- und Jugendchirurgie,02/3 Kinder- und Jugendpsychiatrie,03 Chirurgie,04 Neurochirurgie,05 Innere Medizin,06 Frauenheilkunde und Geburtshilfe,07 Neurologie,08 Psychiatrie,09 Haut- und Geschlechtskrankheiten,10 Augenheilkunde,"11 Hals-, Nasen- und Ohrenheilkunde",12 Urologie,13 Plastische Chirurgie,15 Orthopädie und Traumatologie,"16 Mund-, Kiefer- und Gesichtschirurgie","17 Zahn-, Mund- und Kieferheilkunde",18 Strahlentherapie-Radioonkologie,19 Nuklearmedizinische Therapie,50 Gemischter Belag,61/1 Psychosomatik/Erwachsene,61/2 Psychosomatik/KJ,62 Akutgeriatrie/Remobilisation,63 Remobilisation/Nachsorge,64/1 Palliativmedizin/Erwachsene,64/2 Palliativmedizin/KJ
0,30.0,16.0,NaN,NaN,36.0,NaN,90.0,25.0,34.0,41.0,NaN,NaN,13.0,NaN,NaN,48.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,4.0,NaN
1,10.0,NaN,NaN,NaN,39.0,NaN,37.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,NaN,NaN,NaN,30.0,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.0,NaN,NaN,NaN,29.0,NaN,49.0,23.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,22.0,21.0,NaN,NaN,32.0,NaN,61.0,25.0,38.0,NaN,NaN,NaN,4.0,31.0,NaN,47.0,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,8.0,NaN


In [40]:
hospital_beds = df_hospitals_beds.sum(axis=1)

hospital_beds.head()

0    352.0
1    109.0
2     95.0
3    122.0
4    300.0
dtype: float64

In [41]:
df_hospitals = df_hospitals_info.copy()

df_hospitals['Bettanzahl'] = hospital_beds.astype(int)
df_hospitals['Intensivbette'] = df_hospitals_beds['01 Anästhesiologie und Intensivmedizin/Intensivversorgung'].replace(np.nan, 0).astype(int)

df_hospitals.head()

,Name,PLZ,Bundesland,Bettanzahl,Intensivbette
0,Eisenstadt BBR KH,7000,Burgenland,352,30
1,Güssing LKH,7540,Burgenland,109,10
2,Kittsee LKH,2421,Burgenland,95,10
3,Oberpullendorf LKH,7350,Burgenland,122,12
4,Oberwart LKH,7400,Burgenland,300,22


In [42]:
df_hospitals.sort_values(by='Bettanzahl', ascending=False).head()

,Name,PLZ,Bundesland,Bettanzahl,Intensivbette
153,Wien AKH / Wien AKH,1090,Wien,1532,249
93,Graz LKH,8036,Steiermark,1465,167
130,Innsbruck LKH,6020,Tirol,1435,131
7,Klagenfurt LKH / Klagenfurt,9020,Kärnten,1123,107
58,Wels-Grieskirchen KL / Wels,4600,Oberoesterreich,987,45


In [43]:
df_hospitals.to_csv('../data/cleaned/akutversorgungskrankenhaeuser.csv', index=False)

# 03 - Bevölkerungsanzahl

In [44]:
df_population = pd.read_csv('../data/raw/Bevölkerungszahl.csv', sep=';')

df_population

,Bundesland,01.01.2018,01.01.2019,01.01.2020,01.01.2021,01.01.2022,01.07.2022
0,Österreich,8822267,8858775,8901064,8932664,8978929,9061848
1,Burgenland,292675,293433,294436,296010,297583,300457
2,Kärnten,560898,560939,561293,562089,564513,567644
3,Niederösterreich,1670668,1677542,1684287,1690879,1698796,1713465
4,Oberösterreich,1473576,1482095,1490279,1495608,1505140,1517477
5,Salzburg,552579,555221,558410,560710,562606,566751
6,Steiermark,1240214,1243052,1246395,1247077,1252922,1260417
7,Tirol,751140,754705,757634,760105,764102,768443
8,Vorarlberg,391741,394297,397139,399237,401674,404415
9,Wien,1888776,1897491,1911191,1920949,1931593,1962779


In [45]:
df_population = df_population[['Bundesland', '01.07.2022']]

df_population

,Bundesland,01.07.2022
0,Österreich,9061848
1,Burgenland,300457
2,Kärnten,567644
3,Niederösterreich,1713465
4,Oberösterreich,1517477
5,Salzburg,566751
6,Steiermark,1260417
7,Tirol,768443
8,Vorarlberg,404415
9,Wien,1962779


In [46]:
df_population = df_population.rename(columns={'01.07.2022': 'Bevoelkerungsstand'})

df_population

,Bundesland,Bevoelkerungsstand
0,Österreich,9061848
1,Burgenland,300457
2,Kärnten,567644
3,Niederösterreich,1713465
4,Oberösterreich,1517477
5,Salzburg,566751
6,Steiermark,1260417
7,Tirol,768443
8,Vorarlberg,404415
9,Wien,1962779


In [47]:
df_population.loc[df_population.Bundesland == 'Niederösterreich', 'Bundesland'] = 'Niederoesterreich'
df_population.loc[df_population.Bundesland == 'Oberösterreich', 'Bundesland'] = 'Oberoesterreich'
df_population.loc[df_population.Bundesland == 'Österreich', 'Bundesland'] = 'Oesterreich'



df_population

,Bundesland,Bevoelkerungsstand
0,Oesterreich,9061848
1,Burgenland,300457
2,Kärnten,567644
3,Niederoesterreich,1713465
4,Oberoesterreich,1517477
5,Salzburg,566751
6,Steiermark,1260417
7,Tirol,768443
8,Vorarlberg,404415
9,Wien,1962779


In [48]:
df_population.to_csv('../data/cleaned/bevoelkerungsstand.csv', index=False)

# 04 - Europa hospital beds data set

In [86]:
df_europe_hospitals = pd.read_excel('../data/raw/Hospital_data_europe.xlsx', sheet_name='Data (pivoted)')

df_europe_hospitals.head()

,Measure code,SEX,COUNTRY_REGION,1970,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,HFA_478,ALL,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329.08,359.76,344.83,340.82,340.90,334.24,259.96,289.15,284.56,276.60,278.06,275.33,293.71,295.87,284.58,275.98,271.48,274.52,273.11,271.31,NaN,267.67,275.18,264.46,262.77,263.10,NaN,NaN
1,HFA_478,ALL,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.99,223.44,233.26,244.22,244.28,254.21,283.17,253.80,209.12,200.74,195.28,194.70,NaN,189.84,NaN,NaN,NaN,NaN,NaN,NaN
2,HFA_478,ALL,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,717.55,736.75,748.58,741.67,750.65,770.28,767.57,802.0,832.50,819.49,825.34,772.08,757.00,797.73,787.47,797.81,754.68,710.35,701.90,650.60,574.82,440.45,378.10,385.92,388.11,377.62,369.28,334.51,310.66,300.49,302.64,305.19,330.01,329.82,341.07,NaN
3,HFA_478,ALL,AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.56,828.10,809.78,787.15,766.35,753.05,749.91,741.31,729.50,721.44,707.34,694.62,684.87,675.24,666.67,655.89,649.19,642.77,637.15,637.98,629.26,622.99,616.00,610.23,599.80,587.36,579.96,NaN
4,HFA_478,ALL,AZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,732.98,764.77,786.73,781.05,751.37,750.89,738.52,721.70,704.81,694.97,681.55,673.02,665.12,655.34,649.47,650.40,626.38,627.71,606.88,595.92,382.98,344.14,349.13,349.89,354.18,NaN


In [87]:
df_europe_hospitals.columns.map(type)

Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>],
      dtype='object')

In [88]:
df_europe_hospitals.columns = df_europe_hospitals.columns.astype(str)

In [89]:
df_europe_hospitals.columns.map(type)

Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>],
      dtype='object')

In [90]:
df_europe_hospitals = df_europe_hospitals.rename(columns=lambda x: x.strip())

In [91]:
country_codes = ['AUT', 'FRA', 'DEU', 'CHE', 'CZE']

df_europe_hospitals = df_europe_hospitals.query('COUNTRY_REGION.isin(@country_codes)').copy()

df_europe_hospitals

,Measure code,SEX,COUNTRY_REGION,1970,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
3,HFA_478,ALL,AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.56,828.1,809.78,787.15,766.35,753.05,749.91,741.31,729.50,721.44,707.34,694.62,684.87,675.24,666.67,655.89,649.19,642.77,637.15,637.98,629.26,622.99,616.00,610.23,599.80,587.36,579.96,NaN
10,HFA_478,ALL,CZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,810.07,NaN,NaN,NaN,NaN,817.15,NaN,NaN,NaN,NaN,807.89,NaN,NaN,NaN,NaN,686.21,655.84,631.20,609.94,584.73,577.27,572.11,568.91,557.93,542.85,535.34,526.31,519.75,507.96,498.40,486.17,468.78,452.75,430.87,425.03,NaN
14,HFA_478,ALL,FRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,565.41,550.12,544.06,525.73,513.33,504.77,492.06,482.92,476.67,467.37,462.05,454.28,451.13,447.52,443.30,440.27,435.73,428.04,NaN
16,HFA_478,ALL,DEU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,832.12,802.78,774.41,759.22,745.92,725.00,707.39,696.86,688.62,680.94,671.22,663.52,656.69,644.03,635.21,620.12,616.26,612.96,614.77,614.93,613.88,612.17,620.83,620.83,NaN
44,HFA_478,ALL,CHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.03,563.83,530.72,511.62,503.61,492.60,483.39,470.60,453.96,448.30,434.79,426.64,409.35,394.90,391.59,382.01,375.08,NaN


In [92]:
df_europe_hospitals

,Measure code,SEX,COUNTRY_REGION,1970,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
3,HFA_478,ALL,AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.56,828.1,809.78,787.15,766.35,753.05,749.91,741.31,729.50,721.44,707.34,694.62,684.87,675.24,666.67,655.89,649.19,642.77,637.15,637.98,629.26,622.99,616.00,610.23,599.80,587.36,579.96,NaN
10,HFA_478,ALL,CZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,810.07,NaN,NaN,NaN,NaN,817.15,NaN,NaN,NaN,NaN,807.89,NaN,NaN,NaN,NaN,686.21,655.84,631.20,609.94,584.73,577.27,572.11,568.91,557.93,542.85,535.34,526.31,519.75,507.96,498.40,486.17,468.78,452.75,430.87,425.03,NaN
14,HFA_478,ALL,FRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,565.41,550.12,544.06,525.73,513.33,504.77,492.06,482.92,476.67,467.37,462.05,454.28,451.13,447.52,443.30,440.27,435.73,428.04,NaN
16,HFA_478,ALL,DEU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,832.12,802.78,774.41,759.22,745.92,725.00,707.39,696.86,688.62,680.94,671.22,663.52,656.69,644.03,635.21,620.12,616.26,612.96,614.77,614.93,613.88,612.17,620.83,620.83,NaN
44,HFA_478,ALL,CHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.03,563.83,530.72,511.62,503.61,492.60,483.39,470.60,453.96,448.30,434.79,426.64,409.35,394.90,391.59,382.01,375.08,NaN


In [93]:
countries_seires = df_europe_hospitals.COUNTRY_REGION.map({'AUT': 'Austria',
                                                           'CZE': 'Czech Republic', 
                                                           'FRA': 'France', 
                                                           'DEU': 'Germany', 
                                                           'CHE': 'Switzerland'})

countries_seires

3            Austria
10    Czech Republic
14            France
16           Germany
44       Switzerland
Name: COUNTRY_REGION, dtype: object

In [94]:
df_europe_hospitals = df_europe_hospitals.loc[:, '2000': '2014']

df_europe_hospitals

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
3,684.87,675.24,666.67,655.89,649.19,642.77,637.15,637.98,629.26,622.99,616.00,610.23,599.80,587.36,579.96
10,577.27,572.11,568.91,557.93,542.85,535.34,526.31,519.75,507.96,498.40,486.17,468.78,452.75,430.87,425.03
14,525.73,513.33,504.77,492.06,482.92,476.67,467.37,462.05,454.28,451.13,447.52,443.30,440.27,435.73,428.04
16,680.94,671.22,663.52,656.69,644.03,635.21,620.12,616.26,612.96,614.77,614.93,613.88,612.17,620.83,620.83
44,530.72,511.62,503.61,492.60,483.39,470.60,453.96,448.30,434.79,426.64,409.35,394.90,391.59,382.01,375.08


In [97]:
df_europe_hospitals = df_europe_hospitals.applymap(lambda x: x/100)

df_europe_hospitals

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
3,6.8487,6.7524,6.6667,6.5589,6.4919,6.4277,6.3715,6.3798,6.2926,6.2299,6.1600,6.1023,5.9980,5.8736,5.7996
10,5.7727,5.7211,5.6891,5.5793,5.4285,5.3534,5.2631,5.1975,5.0796,4.9840,4.8617,4.6878,4.5275,4.3087,4.2503
14,5.2573,5.1333,5.0477,4.9206,4.8292,4.7667,4.6737,4.6205,4.5428,4.5113,4.4752,4.4330,4.4027,4.3573,4.2804
16,6.8094,6.7122,6.6352,6.5669,6.4403,6.3521,6.2012,6.1626,6.1296,6.1477,6.1493,6.1388,6.1217,6.2083,6.2083
44,5.3072,5.1162,5.0361,4.9260,4.8339,4.7060,4.5396,4.4830,4.3479,4.2664,4.0935,3.9490,3.9159,3.8201,3.7508


In [98]:
df_europe_hospitals = pd.DataFrame(countries_seires).join(df_europe_hospitals)

df_europe_hospitals

,COUNTRY_REGION,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
3,Austria,6.8487,6.7524,6.6667,6.5589,6.4919,6.4277,6.3715,6.3798,6.2926,6.2299,6.1600,6.1023,5.9980,5.8736,5.7996
10,Czech Republic,5.7727,5.7211,5.6891,5.5793,5.4285,5.3534,5.2631,5.1975,5.0796,4.9840,4.8617,4.6878,4.5275,4.3087,4.2503
14,France,5.2573,5.1333,5.0477,4.9206,4.8292,4.7667,4.6737,4.6205,4.5428,4.5113,4.4752,4.4330,4.4027,4.3573,4.2804
16,Germany,6.8094,6.7122,6.6352,6.5669,6.4403,6.3521,6.2012,6.1626,6.1296,6.1477,6.1493,6.1388,6.1217,6.2083,6.2083
44,Switzerland,5.3072,5.1162,5.0361,4.9260,4.8339,4.7060,4.5396,4.4830,4.3479,4.2664,4.0935,3.9490,3.9159,3.8201,3.7508


In [99]:
df_europe_hospitals.to_csv('../data/cleaned/hospital_beds_europe.csv', index=False)

# 05 - Ärtztedaten